# Evaluation of frozen/quantized graph

In [1]:
import os
import cv2
import numpy as np
import urllib

In [2]:
import tensorflow as tf

In [3]:
# Use CPU
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

In [4]:
print("TensorFlow Version: %s" % tf.__version__)

TensorFlow Version: 1.15.0


## WordNet ID vs. words vs. class

- Figure out class No. from WordNet ID

In [5]:
# Class No. vs. word
label_file = 'ImageNetLabels.txt'
if not os.path.exists(label_file):
    dl = urllib.request.urlretrieve(
        'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt',
        label_file)
    
imagenet_labels = np.array(open(label_file).read().splitlines())

In [6]:
# WordNet ID vs. word
words_file = 'words.txt'
if not os.path.exists(words_file):
    dl = urllib.request.urlretrieve(
        'http://image-net.org/archive/words.txt',
        words_file)
    
wn_ids = {}
with open(words_file, 'r') as f:
    for line in f:
        elem = line.split(sep='\t')
        elem[1] = elem[1].replace('\n', '').rsplit(sep=',')[0]
        wn_ids.update( {elem[1]: elem[0]} )

In [7]:
wn_id_to_word = {}
for lb in imagenet_labels:
    wn_id_to_word.update( {wn_ids[lb]: lb} )

## Load model

In [8]:
input_graph_def = tf.Graph().as_graph_def()
input_graph_def.ParseFromString(tf.gfile.GFile('frozen.pb', 'rb').read())

17085200

In [9]:
tf.import_graph_def(input_graph_def, name = '')

In [10]:
# Show node names
# for node in input_graph_def.node:
#     print(node.name)

In [11]:
input_node = 'input'
output_node = 'MobilenetV1/Predictions/Reshape_1'

In [12]:
# Get input & output tensors
images_in = tf.get_default_graph().get_tensor_by_name(input_node+':0')
output = tf.get_default_graph().get_tensor_by_name(output_node+':0')

In [13]:
print('Input  tensor shape: %s' % images_in.shape)
print('Output tensor shape: %s' % output.shape)

Input  tensor shape: (?, 224, 224, 3)
Output tensor shape: (?, 1001)


In [14]:
labels = tf.placeholder(tf.int32, shape=[None, output.shape[1].value])

## Load image

In [15]:
img_path = os.path.join('_calib', 'n01530575_10101.png')

In [16]:
# Load as RGB image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [17]:
# Normalization - ImageNet ver.
# MEAN_RGB = np.reshape(np.asarray([0.485, 0.456, 0.406]), [1, 1, 3])
# STDDEV_RGB = np.reshape(np.asarray([0.229, 0.224, 0.225]), [1, 1, 3])

# x_test = img.astype(np.float32)
# x_test = np.divide((x_test/255.0 - MEAN_RGB), STDDEV_RGB)
# x_test = np.expand_dims(x_test, axis=0)

In [18]:
# Normalization - Xilinx ver.
x_test = img.astype(np.float32)
x_test = 2*(x_test/255.0 - 0.5)
x_test = np.expand_dims(x_test, axis=0)

In [19]:
# Ground truth
y_test = np.zeros(output.shape[1].value)
y_test = np.expand_dims(y_test, axis=0)

In [20]:
# Get WordNet ID
img_id = os.path.split(img_path)[1].split('_')[0]
idx = list(wn_id_to_word).index(img_id)
y_test[0][idx] = 1
# print(idx)

***

## Inference

In [21]:
# top 5 and top 1 accuracy
in_top5 = tf.nn.in_top_k(predictions=output, targets=tf.argmax(labels, 1), k=5)
in_top1 = tf.nn.in_top_k(predictions=output, targets=tf.argmax(labels, 1), k=1)
top5_acc = tf.reduce_mean(tf.cast(in_top5, tf.float32))
top1_acc = tf.reduce_mean(tf.cast(in_top1, tf.float32))

In [22]:
# Create the Computational graph
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())

    feed_dict = {images_in: x_test, labels: y_test}
    t5_acc, t1_acc, logits = sess.run([top5_acc, top1_acc, output], feed_dict)

#     print ('Top 1 accuracy with validation set: {:1.4f}'.format(t1_acc))
#     print ('Top 5 accuracy with validation set: {:1.4f}'.format(t5_acc))

***

## Show result

In [23]:
print ('Top 1 accuracy with validation set: {:1.4f}'.format(t1_acc))
print ('Top 5 accuracy with validation set: {:1.4f}'.format(t5_acc))

Top 1 accuracy with validation set: 1.0000
Top 5 accuracy with validation set: 1.0000


In [24]:
result = np.argmax(logits, axis=1)
for r in result:
    print('%s: %.4f' % (list(wn_id_to_word.values())[r], logits[0][r]) )

brambling: 0.9999
